In [1]:
# python script of M4_weekly and DeepAR

# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator


# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint



INFO:root:Using CPU


In [2]:
dataset = get_dataset("m4_hourly", regenerate=False)
dataset.metadata

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


<MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='414'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=48>

In [21]:
# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=150,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.47it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 11.325 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.44it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 9.198 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 9.327 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.41it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 9.245 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.663846
INFO:root:Epoch[4] Learning rate is 0.001

INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.35it/s, avg_epoch_loss=2.88]
INFO:root:Epoch[74] Elapsed time 9.349 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.881696
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.28it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[75] Elapsed time 9.470 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.684727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.30it/s, avg_epoch_loss=2.82]
INFO:root:Epoch[76] Elapsed time 9.441 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.817950
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.34it/s, avg_epoch_loss=2.83]
INFO:root:Epoch[77] Elapsed time 9.363 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.833692
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.25it/s, avg_epoch_loss=2.59]
INFO:ro

INFO:root:Epoch[146] Elapsed time 9.909 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.687036
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.33it/s, avg_epoch_loss=2.72]
INFO:root:Epoch[147] Elapsed time 9.388 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.723706
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 9.317 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.713427
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.32it/s, avg_epoch_loss=2.36]
INFO:root:Epoch[149] Elapsed time 9.407 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.358524
INFO:root:Loading parameters from best epoch (149)
INFO:root:Final loss: 2.358523907661438 (occurred at epoch 149)
INFO:root:End model training
Running evaluation: 100%|██████████| 414/414 [01:16<00:00,  5.41it/s]


{'Coverage[0.1]': 0.16732085346215758,
 'Coverage[0.2]': 0.23716787439613526,
 'Coverage[0.3]': 0.3008252818035424,
 'Coverage[0.4]': 0.35975241545893705,
 'Coverage[0.5]': 0.4176731078904989,
 'Coverage[0.6]': 0.4681964573268918,
 'Coverage[0.7]': 0.5316525764895329,
 'Coverage[0.8]': 0.6022544283413843,
 'Coverage[0.9]': 0.6835748792270528,
 'MAE_Coverage': 0.10469001610305971,
 'MASE': 1.4927665506982832,
 'MSE': 3784209.672755249,
 'MSIS': 30.277210209495337,
 'ND': 0.04993182184130491,
 'NRMSE': 0.26557703496222385,
 'QuantileLoss[0.1]': 5216897.921738435,
 'QuantileLoss[0.2]': 6399495.502192497,
 'QuantileLoss[0.3]': 7031861.944239711,
 'QuantileLoss[0.4]': 7283580.1222492205,
 'QuantileLoss[0.5]': 7268019.243769169,
 'QuantileLoss[0.6]': 6973998.427048111,
 'QuantileLoss[0.7]': 6456008.270732499,
 'QuantileLoss[0.8]': 5649742.092518616,
 'QuantileLoss[0.9]': 4427660.656889343,
 'RMSE': 1945.3045192861834,
 'abs_error': 7268019.244678497,
 'abs_target_mean': 7324.822041043147,
 '

In [22]:
# seeds
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=150,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.96it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 10.088 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.33it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 9.380 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.32it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 9.402 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.28it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 9.470 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.771905
INFO:root:Epoch[4] Learning rate is 0.001

100%|██████████| 50/50 [00:09<00:00,  5.10it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[73] Elapsed time 9.801 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.904866
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:10<00:00,  4.94it/s, avg_epoch_loss=2.93]
INFO:root:Epoch[74] Elapsed time 10.133 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.926818
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.29it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[75] Elapsed time 9.460 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.903587
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.30it/s, avg_epoch_loss=2.98]
INFO:root:Epoch[76] Elapsed time 9.434 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.979229
INFO:root:Loading parameters from best epoch (66)
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:09<00:00,  5.27it/s, avg_epoch_loss=

INFO:root:Epoch[146] Elapsed time 9.603 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.748752
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.27it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[147] Elapsed time 9.497 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.971499
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  5.00it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[148] Elapsed time 10.016 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.970719
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.30it/s, avg_epoch_loss=2.76]
INFO:root:Epoch[149] Elapsed time 9.440 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.764020
INFO:root:Loading parameters from best epoch (117)
INFO:root:Final loss: 2.590440773963928 (occurred at epoch 117)
INFO:root:End model training
Running evaluation: 100%|██████████| 414/414 [01:15<00:00,  5.48it/s]


{'Coverage[0.1]': 0.1930857487922704,
 'Coverage[0.2]': 0.2636876006441225,
 'Coverage[0.3]': 0.3250805152979066,
 'Coverage[0.4]': 0.38466183574879226,
 'Coverage[0.5]': 0.4413244766505636,
 'Coverage[0.6]': 0.49642713365539426,
 'Coverage[0.7]': 0.5575181159420292,
 'Coverage[0.8]': 0.62686191626409,
 'Coverage[0.9]': 0.7084842995169082,
 'MAE_Coverage': 0.096286231884058,
 'MASE': 1.503435116375025,
 'MSE': 6450557.939593869,
 'MSIS': 28.90898598238708,
 'ND': 0.060993915225668084,
 'NRMSE': 0.3467380978655547,
 'QuantileLoss[0.1]': 8072567.560259532,
 'QuantileLoss[0.2]': 9294333.93619461,
 'QuantileLoss[0.3]': 9608928.768697357,
 'QuantileLoss[0.4]': 9418854.167272568,
 'QuantileLoss[0.5]': 8878205.07075119,
 'QuantileLoss[0.6]': 7986753.475995828,
 'QuantileLoss[0.7]': 6825970.130126381,
 'QuantileLoss[0.8]': 5342446.02446289,
 'QuantileLoss[0.9]': 3449465.8621416087,
 'RMSE': 2539.794861714991,
 'abs_error': 8878204.986739159,
 'abs_target_mean': 7324.822041043147,
 'abs_target_

In [23]:
# seeds
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=150,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.96it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[0] Elapsed time 10.085 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.111396
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.26it/s, avg_epoch_loss=4.26]
INFO:root:Epoch[1] Elapsed time 9.505 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.256519
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.18it/s, avg_epoch_loss=4.27]
INFO:root:Epoch[2] Elapsed time 9.654 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.270225
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.21it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[3] Elapsed time 9.615 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.050069
INFO:root:Epoch[4] Learning rate is 0.001

INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.29it/s, avg_epoch_loss=2.92]
INFO:root:Epoch[74] Elapsed time 9.461 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.916431
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.50it/s, avg_epoch_loss=2.89]
INFO:root:Epoch[75] Elapsed time 9.090 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.886410
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.48it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[76] Elapsed time 9.128 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.453312
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.24it/s, avg_epoch_loss=2.95]
INFO:root:Epoch[77] Elapsed time 9.547 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.951038
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.27it/s, avg_epoch_loss=2.84]
INFO:ro

INFO:root:Epoch[146] Elapsed time 9.081 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.550389
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.43it/s, avg_epoch_loss=2.4]
INFO:root:Epoch[147] Elapsed time 9.223 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.403520
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.44it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 9.198 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.707970
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.55it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[149] Elapsed time 9.015 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.774012
INFO:root:Loading parameters from best epoch (147)
INFO:root:Final loss: 2.403519561290741 (occurred at epoch 147)
INFO:root:End model training
Running evaluation: 100%|██████████| 414/414 [01:14<00:00,  5.59it/s]


{'Coverage[0.1]': 0.19786634460547498,
 'Coverage[0.2]': 0.2730978260869563,
 'Coverage[0.3]': 0.3343397745571659,
 'Coverage[0.4]': 0.39185789049919484,
 'Coverage[0.5]': 0.44504830917874405,
 'Coverage[0.6]': 0.4934581320450887,
 'Coverage[0.7]': 0.5505233494363931,
 'Coverage[0.8]': 0.6177033011272139,
 'Coverage[0.9]': 0.6971115136876004,
 'MAE_Coverage': 0.10106682769726245,
 'MASE': 1.560219301118818,
 'MSE': 56500482.74376654,
 'MSIS': 34.26988815181001,
 'ND': 0.13921045185738232,
 'NRMSE': 1.0261928902350617,
 'QuantileLoss[0.1]': 6117694.0092194555,
 'QuantileLoss[0.2]': 10182985.7061388,
 'QuantileLoss[0.3]': 13839980.51817646,
 'QuantileLoss[0.4]': 17184782.583482742,
 'QuantileLoss[0.5]': 20263315.363336563,
 'QuantileLoss[0.6]': 23113280.00223007,
 'QuantileLoss[0.7]': 25590558.884727668,
 'QuantileLoss[0.8]': 27607924.171578597,
 'QuantileLoss[0.9]': 28820208.148647882,
 'RMSE': 7516.68030075555,
 'abs_error': 20263315.1735487,
 'abs_target_mean': 7324.822041043147,
 'ab

In [24]:
# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=200,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.04it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 9.922 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.48it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 9.121 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.50it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 9.100 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.44it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 9.205 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.663846
INFO:root:Epoch[4] Learning rate is 0.001


INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.14it/s, avg_epoch_loss=2.88]
INFO:root:Epoch[74] Elapsed time 9.728 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.881696
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.42it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[75] Elapsed time 9.233 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.684727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.45it/s, avg_epoch_loss=2.82]
INFO:root:Epoch[76] Elapsed time 9.183 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.817950
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.47it/s, avg_epoch_loss=2.83]
INFO:root:Epoch[77] Elapsed time 9.143 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.833692
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.41it/s, avg_epoch_loss=2.59]
INFO:ro

INFO:root:Epoch[146] Elapsed time 9.188 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.687036
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.49it/s, avg_epoch_loss=2.72]
INFO:root:Epoch[147] Elapsed time 9.115 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.723706
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.54it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 9.034 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.713427
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.48it/s, avg_epoch_loss=2.36]
INFO:root:Epoch[149] Elapsed time 9.135 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.358524
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:08<00:00,  5.57it/s, avg_epoch_loss=2.57]
INFO:root:Epoch[150] Elapsed time 8.984 seconds
INFO:root:Epoch[150] Evaluation metric 'epoch_loss'=2.566216
INF

{'Coverage[0.1]': 0.19424315619967814,
 'Coverage[0.2]': 0.2687198067632852,
 'Coverage[0.3]': 0.3336352657004831,
 'Coverage[0.4]': 0.3915056360708534,
 'Coverage[0.5]': 0.44907407407407396,
 'Coverage[0.6]': 0.5039754428341386,
 'Coverage[0.7]': 0.5671296296296294,
 'Coverage[0.8]': 0.6341586151368759,
 'Coverage[0.9]': 0.7124597423510469,
 'MAE_Coverage': 0.09314389872964758,
 'MASE': 1.3969220822040853,
 'MSE': 3839147.731909421,
 'MSIS': 27.981724409807438,
 'ND': 0.050207332135630715,
 'NRMSE': 0.2674978737450292,
 'QuantileLoss[0.1]': 5359931.857766247,
 'QuantileLoss[0.2]': 6503575.588689232,
 'QuantileLoss[0.3]': 7100133.545887088,
 'QuantileLoss[0.4]': 7341797.838847351,
 'QuantileLoss[0.5]': 7308122.246352673,
 'QuantileLoss[0.6]': 7017300.127007866,
 'QuantileLoss[0.7]': 6494237.485284423,
 'QuantileLoss[0.8]': 5685456.045336151,
 'QuantileLoss[0.9]': 4469989.325927543,
 'RMSE': 1959.3743215397667,
 'abs_error': 7308122.210030556,
 'abs_target_mean': 7324.822041043147,
 'ab

In [25]:
# seeds
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=200,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.96it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 10.082 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.19it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 9.640 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.33it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 9.395 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.32it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 9.409 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.771905
INFO:root:Epoch[4] Learning rate is 0.001

100%|██████████| 50/50 [00:09<00:00,  5.38it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[73] Elapsed time 9.305 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.904866
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.38it/s, avg_epoch_loss=2.93]
INFO:root:Epoch[74] Elapsed time 9.298 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.926818
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.43it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[75] Elapsed time 9.220 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.903587
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.44it/s, avg_epoch_loss=2.98]
INFO:root:Epoch[76] Elapsed time 9.202 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.979229
INFO:root:Loading parameters from best epoch (66)
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:09<00:00,  5.42it/s, avg_epoch_loss=3

INFO:root:Epoch[146] Elapsed time 10.008 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.748752
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.02it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[147] Elapsed time 9.976 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.971499
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.54it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[148] Elapsed time 11.031 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.970719
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.86it/s, avg_epoch_loss=2.76]
INFO:root:Epoch[149] Elapsed time 10.288 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.764020
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.63it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[150] Elapsed time 10.809 seconds
INFO:root:Epoch[150] Evaluation metric 'epoch_loss'=2.675828

{'Coverage[0.1]': 0.22921698872785828,
 'Coverage[0.2]': 0.3011272141706923,
 'Coverage[0.3]': 0.363778180354267,
 'Coverage[0.4]': 0.42441626409017696,
 'Coverage[0.5]': 0.48384661835748793,
 'Coverage[0.6]': 0.5344706119162637,
 'Coverage[0.7]': 0.5964170692431566,
 'Coverage[0.8]': 0.6576590177133653,
 'Coverage[0.9]': 0.7395330112721417,
 'MAE_Coverage': 0.08962359098228659,
 'MASE': 1.4291122602719997,
 'MSE': 12206842.09424923,
 'MSIS': 27.87025303678336,
 'ND': 0.07694593431365043,
 'NRMSE': 0.4769848503341247,
 'QuantileLoss[0.1]': 12564101.999128724,
 'QuantileLoss[0.2]': 13214923.77512932,
 'QuantileLoss[0.3]': 12992104.162103271,
 'QuantileLoss[0.4]': 12284570.15846138,
 'QuantileLoss[0.5]': 11200162.744219303,
 'QuantileLoss[0.6]': 9752148.03690033,
 'QuantileLoss[0.7]': 8066107.829995918,
 'QuantileLoss[0.8]': 6067529.345143508,
 'QuantileLoss[0.9]': 3683899.3749513617,
 'RMSE': 3493.829144971063,
 'abs_error': 11200162.757305145,
 'abs_target_mean': 7324.822041043147,
 'a

In [26]:
# seeds
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=200,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.62it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[0] Elapsed time 13.848 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.111396
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.69it/s, avg_epoch_loss=4.26]
INFO:root:Epoch[1] Elapsed time 13.565 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.256519
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.12it/s, avg_epoch_loss=4.27]
INFO:root:Epoch[2] Elapsed time 12.131 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.270225
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.92it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[3] Elapsed time 12.762 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.050069
INFO:root:Epoch[4] Learning rate is 0.

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.948231
INFO:root:Loading parameters from best epoch (63)
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:14<00:00,  3.47it/s, avg_epoch_loss=2.92]
INFO:root:Epoch[74] Elapsed time 14.423 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.916431
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:12<00:00,  4.17it/s, avg_epoch_loss=2.89]
INFO:root:Epoch[75] Elapsed time 12.010 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.886410
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:11<00:00,  4.19it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[76] Elapsed time 11.956 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.453312
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:12<00:00,  3.94it/s, avg_epoch_loss=2.95]
INFO:root:Epoch[77] Elapsed time 12.693 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.951038
INFO:r

INFO:root:Epoch[145] Evaluation metric 'epoch_loss'=2.718552
INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.21it/s, avg_epoch_loss=2.55]
INFO:root:Epoch[146] Elapsed time 11.883 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.550389
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.25it/s, avg_epoch_loss=2.4] 
INFO:root:Epoch[147] Elapsed time 11.783 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.403520
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.18it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 11.963 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.707970
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.97it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[149] Elapsed time 10.084 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.774012
INFO:root:Epoch[150] Learning rate is 5e-05
100

{'Coverage[0.1]': 0.20964170692431555,
 'Coverage[0.2]': 0.2806964573268922,
 'Coverage[0.3]': 0.34113325281803536,
 'Coverage[0.4]': 0.39689009661835767,
 'Coverage[0.5]': 0.448520531400966,
 'Coverage[0.6]': 0.49501811594202894,
 'Coverage[0.7]': 0.5537439613526574,
 'Coverage[0.8]': 0.6190619967793879,
 'Coverage[0.9]': 0.696256038647343,
 'MAE_Coverage': 0.1024422973698336,
 'MASE': 1.560594300956167,
 'MSE': 61654738.61067279,
 'MSIS': 35.22681002767355,
 'ND': 0.14334861852229547,
 'NRMSE': 1.0719786913863227,
 'QuantileLoss[0.1]': 6091817.731376171,
 'QuantileLoss[0.2]': 10287948.424474526,
 'QuantileLoss[0.3]': 14097892.847989561,
 'QuantileLoss[0.4]': 17612344.171542358,
 'QuantileLoss[0.5]': 20865661.670767784,
 'QuantileLoss[0.6]': 23901654.419775773,
 'QuantileLoss[0.7]': 26587815.108136177,
 'QuantileLoss[0.8]': 28861422.557629395,
 'QuantileLoss[0.9]': 30393165.422930527,
 'RMSE': 7852.053146195127,
 'abs_error': 20865662.010679245,
 'abs_target_mean': 7324.822041043147,


In [27]:
# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=250,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:14<00:00,  3.39it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 14.745 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.71it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 10.618 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.08it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 9.843 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.86it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 10.286 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.663846
INFO:root:Epoch[4] Learning rate is 0.0

INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.94it/s, avg_epoch_loss=3.19]
INFO:root:Epoch[37] Elapsed time 12.692 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=3.188947
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.73it/s, avg_epoch_loss=2.91]
INFO:root:Epoch[38] Elapsed time 13.441 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=2.912309
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.83it/s, avg_epoch_loss=3.09]
INFO:root:Epoch[39] Elapsed time 13.054 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=3.094895
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.68it/s, avg_epoch_loss=3.15]
INFO:root:Epoch[40] Elapsed time 13.588 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=3.147061
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.19it/s, avg_epoch_loss=3.2] 
INFO:roo

INFO:root:Epoch[110] Learning rate is 0.000125
100%|██████████| 50/50 [00:12<00:00,  4.11it/s, avg_epoch_loss=2.6] 
INFO:root:Epoch[110] Elapsed time 12.163 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=2.597788
INFO:root:Epoch[111] Learning rate is 0.000125
100%|██████████| 50/50 [00:13<00:00,  3.80it/s, avg_epoch_loss=2.53]
INFO:root:Epoch[111] Elapsed time 13.166 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=2.525449
INFO:root:Epoch[112] Learning rate is 0.000125
100%|██████████| 50/50 [00:14<00:00,  3.45it/s, avg_epoch_loss=2.63]
INFO:root:Epoch[112] Elapsed time 14.483 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=2.634202
INFO:root:Epoch[113] Learning rate is 0.000125
100%|██████████| 50/50 [00:14<00:00,  3.41it/s, avg_epoch_loss=2.69]
INFO:root:Epoch[113] Elapsed time 14.667 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=2.688214
INFO:root:Epoch[114] Learning rate is 0.000125
100%|██████████| 50/50 [00:11<00:00,  4.17it/s, a

INFO:root:Epoch[182] Evaluation metric 'epoch_loss'=2.714263
INFO:root:Epoch[183] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.56it/s, avg_epoch_loss=2.8] 
INFO:root:Epoch[183] Elapsed time 10.969 seconds
INFO:root:Epoch[183] Evaluation metric 'epoch_loss'=2.796406
INFO:root:Epoch[184] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.06it/s, avg_epoch_loss=2.66]
INFO:root:Epoch[184] Elapsed time 9.895 seconds
INFO:root:Epoch[184] Evaluation metric 'epoch_loss'=2.663109
INFO:root:Epoch[185] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.83it/s, avg_epoch_loss=2.66]
INFO:root:Epoch[185] Elapsed time 10.349 seconds
INFO:root:Epoch[185] Evaluation metric 'epoch_loss'=2.658560
INFO:root:Epoch[186] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.19it/s, avg_epoch_loss=2.37]
INFO:root:Epoch[186] Elapsed time 9.649 seconds
INFO:root:Epoch[186] Evaluation metric 'epoch_loss'=2.372812
INFO:root:Epoch[187] Learning rate is 5e-05
100%|

{'Coverage[0.1]': 0.19379025764895352,
 'Coverage[0.2]': 0.26841787439613546,
 'Coverage[0.3]': 0.3330817230273751,
 'Coverage[0.4]': 0.39170692431562,
 'Coverage[0.5]': 0.45013083735909826,
 'Coverage[0.6]': 0.5027677133655396,
 'Coverage[0.7]': 0.5672805958132042,
 'Coverage[0.8]': 0.6343599033816425,
 'Coverage[0.9]': 0.7118558776167472,
 'MAE_Coverage': 0.09302088924673471,
 'MASE': 1.3963500962119446,
 'MSE': 3853045.6435174835,
 'MSIS': 27.944284640316297,
 'ND': 0.05025934135488982,
 'NRMSE': 0.26798161435734597,
 'QuantileLoss[0.1]': 5399082.933024502,
 'QuantileLoss[0.2]': 6530931.899779129,
 'QuantileLoss[0.3]': 7115432.147768307,
 'QuantileLoss[0.4]': 7350000.557302093,
 'QuantileLoss[0.5]': 7315692.634308815,
 'QuantileLoss[0.6]': 7024245.945386886,
 'QuantileLoss[0.7]': 6503186.7651197435,
 'QuantileLoss[0.8]': 5697369.5483963,
 'QuantileLoss[0.9]': 4481590.631687545,
 'RMSE': 1962.9176354390124,
 'abs_error': 7315692.612882614,
 'abs_target_mean': 7324.822041043147,
 'abs

In [28]:
# seeds
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=250,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.13it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 9.744 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.42it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 9.224 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.42it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 9.227 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 9.320 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.771905
INFO:root:Epoch[4] Learning rate is 0.001


100%|██████████| 50/50 [00:09<00:00,  5.45it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[73] Elapsed time 9.183 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.904866
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.15it/s, avg_epoch_loss=2.93]
INFO:root:Epoch[74] Elapsed time 9.718 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.926818
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:11<00:00,  4.49it/s, avg_epoch_loss=2.9] 
INFO:root:Epoch[75] Elapsed time 11.142 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.903587
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:10<00:00,  4.66it/s, avg_epoch_loss=2.98]
INFO:root:Epoch[76] Elapsed time 10.731 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.979229
INFO:root:Loading parameters from best epoch (66)
INFO:root:Epoch[77] Learning rate is 6.25e-05
100%|██████████| 50/50 [00:10<00:00,  4.60it/s, avg_epoch_los

100%|██████████| 50/50 [00:12<00:00,  4.13it/s, avg_epoch_loss=2.75]
INFO:root:Epoch[146] Elapsed time 12.107 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.748752
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.57it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[147] Elapsed time 10.978 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.971499
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.02it/s, avg_epoch_loss=2.97]
INFO:root:Epoch[148] Elapsed time 9.973 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.970719
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.81it/s, avg_epoch_loss=2.76]
INFO:root:Epoch[149] Elapsed time 10.417 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.764020
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:13<00:00,  3.77it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[150] Elapsed time 13.256

INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=2.788302
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.38it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[220] Elapsed time 9.305 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=2.677472
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.34it/s, avg_epoch_loss=2.78]
INFO:root:Epoch[221] Elapsed time 9.368 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=2.781995
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.49it/s, avg_epoch_loss=2.48]
INFO:root:Epoch[222] Elapsed time 9.112 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=2.483906
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.14it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[223] Elapsed time 9.742 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=2.767352
INFO:root:Epoch[224] Learning rate is 5e-05
100%|██

{'Coverage[0.1]': 0.22901570048309178,
 'Coverage[0.2]': 0.3007246376811593,
 'Coverage[0.3]': 0.3650362318840579,
 'Coverage[0.4]': 0.4258756038647343,
 'Coverage[0.5]': 0.48263888888888884,
 'Coverage[0.6]': 0.5339673913043476,
 'Coverage[0.7]': 0.5960144927536233,
 'Coverage[0.8]': 0.6575583735909817,
 'Coverage[0.9]': 0.7394323671497586,
 'MAE_Coverage': 0.09011562891393815,
 'MASE': 1.42773669680761,
 'MSE': 12210717.382521335,
 'MSIS': 27.73252807738134,
 'ND': 0.07705210353711792,
 'NRMSE': 0.4770605581666256,
 'QuantileLoss[0.1]': 12585632.143371964,
 'QuantileLoss[0.2]': 13224637.239519121,
 'QuantileLoss[0.3]': 13007027.259612177,
 'QuantileLoss[0.4]': 12308213.10334282,
 'QuantileLoss[0.5]': 11215616.657608986,
 'QuantileLoss[0.6]': 9769625.83834877,
 'QuantileLoss[0.7]': 8082369.48792572,
 'QuantileLoss[0.8]': 6070595.112804031,
 'QuantileLoss[0.9]': 3682751.2695421204,
 'RMSE': 3494.3836913712457,
 'abs_error': 11215616.628822327,
 'abs_target_mean': 7324.822041043147,
 'a

In [29]:
# seeds
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=250,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.64it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[0] Elapsed time 10.775 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.111396
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.04it/s, avg_epoch_loss=4.26]
INFO:root:Epoch[1] Elapsed time 9.930 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.256519
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.15it/s, avg_epoch_loss=4.27]
INFO:root:Epoch[2] Elapsed time 9.724 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.270225
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.16it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[3] Elapsed time 9.687 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=4.050069
INFO:root:Epoch[4] Learning rate is 0.001

INFO:root:Loading parameters from best epoch (63)
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:12<00:00,  3.85it/s, avg_epoch_loss=2.92]
INFO:root:Epoch[74] Elapsed time 12.991 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.916431
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:11<00:00,  4.37it/s, avg_epoch_loss=2.89]
INFO:root:Epoch[75] Elapsed time 11.460 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.886410
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:12<00:00,  4.14it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[76] Elapsed time 12.077 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.453312
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:11<00:00,  4.30it/s, avg_epoch_loss=2.95]
INFO:root:Epoch[77] Elapsed time 11.651 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.951038
INFO:root:Epoch[78] Learning rate is 0.0005
100%|██████████| 50/50

INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 50/50 [00:13<00:00,  3.81it/s, avg_epoch_loss=2.55]
INFO:root:Epoch[146] Elapsed time 13.134 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.550389
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.96it/s, avg_epoch_loss=2.4]
INFO:root:Epoch[147] Elapsed time 10.081 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.403520
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.25it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 9.521 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.707970
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.31it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[149] Elapsed time 9.422 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.774012
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.25it/s, avg_epoch_loss=2.65

INFO:root:Epoch[219] Elapsed time 10.658 seconds
INFO:root:Epoch[219] Evaluation metric 'epoch_loss'=2.817164
INFO:root:Epoch[220] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.57it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[220] Elapsed time 10.936 seconds
INFO:root:Epoch[220] Evaluation metric 'epoch_loss'=2.769871
INFO:root:Epoch[221] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.10it/s, avg_epoch_loss=2.77]
INFO:root:Epoch[221] Elapsed time 9.815 seconds
INFO:root:Epoch[221] Evaluation metric 'epoch_loss'=2.769806
INFO:root:Epoch[222] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.29it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[222] Elapsed time 9.463 seconds
INFO:root:Epoch[222] Evaluation metric 'epoch_loss'=2.675332
INFO:root:Epoch[223] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.47it/s, avg_epoch_loss=2.42]
INFO:root:Epoch[223] Elapsed time 9.152 seconds
INFO:root:Epoch[223] Evaluation metric 'epoch_loss'=2.423464
I

{'Coverage[0.1]': 0.20944041867954893,
 'Coverage[0.2]': 0.2808474235104672,
 'Coverage[0.3]': 0.3422403381642511,
 'Coverage[0.4]': 0.39709138486312423,
 'Coverage[0.5]': 0.44841988727858273,
 'Coverage[0.6]': 0.4956219806763285,
 'Coverage[0.7]': 0.5536936392914656,
 'Coverage[0.8]': 0.6187097423510463,
 'Coverage[0.9]': 0.6960044283413852,
 'MAE_Coverage': 0.10255412417248162,
 'MASE': 1.560502804054904,
 'MSE': 61630427.071008496,
 'MSIS': 35.20109602267023,
 'ND': 0.14326681058656612,
 'NRMSE': 1.0717673205923637,
 'QuantileLoss[0.1]': 6076939.562150955,
 'QuantileLoss[0.2]': 10279489.13983059,
 'QuantileLoss[0.3]': 14087799.861387443,
 'QuantileLoss[0.4]': 17599719.918413162,
 'QuantileLoss[0.5]': 20853753.776834488,
 'QuantileLoss[0.6]': 23884500.85460167,
 'QuantileLoss[0.7]': 26567309.214417648,
 'QuantileLoss[0.8]': 28845945.591121294,
 'QuantileLoss[0.9]': 30376663.38704338,
 'RMSE': 7850.5048927447015,
 'abs_error': 20853754.14052105,
 'abs_target_mean': 7324.822041043147,


In [3]:
# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=500,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:18<00:00,  2.67it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 18.701 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.21it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 15.648 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:19<00:00,  2.56it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 19.516 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:18<00:00,  2.71it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 18.492 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.663846
INFO:root:Epoch[4] Learning rate is 0.

INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=3.246917
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:17<00:00,  2.89it/s, avg_epoch_loss=3.19]
INFO:root:Epoch[37] Elapsed time 17.340 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=3.188947
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.81it/s, avg_epoch_loss=2.91]
INFO:root:Epoch[38] Elapsed time 13.123 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=2.912309
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.01it/s, avg_epoch_loss=3.09]
INFO:root:Epoch[39] Elapsed time 12.477 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=3.094895
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.32it/s, avg_epoch_loss=3.15]
INFO:root:Epoch[40] Elapsed time 11.577 seconds
INFO:root:Epoch[40] Evaluation metric 'epoch_loss'=3.147061
INFO:root:Epoch[41] Learning rate is 0.001
100%|██████████| 

INFO:root:Epoch[73] Elapsed time 11.935 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.648038
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.82it/s, avg_epoch_loss=2.88]
INFO:root:Epoch[74] Elapsed time 10.376 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.881696
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[75] Elapsed time 9.312 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.684727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.32it/s, avg_epoch_loss=2.82]
INFO:root:Epoch[76] Elapsed time 9.413 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.817950
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.40it/s, avg_epoch_loss=2.83]
INFO:root:Epoch[77] Elapsed time 9.262 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.833692
INFO:root:E

INFO:root:Epoch[110] Learning rate is 0.000125
100%|██████████| 50/50 [00:11<00:00,  4.53it/s, avg_epoch_loss=2.6] 
INFO:root:Epoch[110] Elapsed time 11.040 seconds
INFO:root:Epoch[110] Evaluation metric 'epoch_loss'=2.597788
INFO:root:Epoch[111] Learning rate is 0.000125
100%|██████████| 50/50 [00:10<00:00,  4.59it/s, avg_epoch_loss=2.53]
INFO:root:Epoch[111] Elapsed time 10.890 seconds
INFO:root:Epoch[111] Evaluation metric 'epoch_loss'=2.525449
INFO:root:Epoch[112] Learning rate is 0.000125
100%|██████████| 50/50 [00:12<00:00,  4.12it/s, avg_epoch_loss=2.63]
INFO:root:Epoch[112] Elapsed time 12.159 seconds
INFO:root:Epoch[112] Evaluation metric 'epoch_loss'=2.634202
INFO:root:Epoch[113] Learning rate is 0.000125
100%|██████████| 50/50 [00:14<00:00,  3.48it/s, avg_epoch_loss=2.69]
INFO:root:Epoch[113] Elapsed time 14.366 seconds
INFO:root:Epoch[113] Evaluation metric 'epoch_loss'=2.688214
INFO:root:Epoch[114] Learning rate is 0.000125
100%|██████████| 50/50 [00:13<00:00,  3.82it/s, a

INFO:root:Epoch[146] Learning rate is 5e-05
100%|██████████| 50/50 [00:09<00:00,  5.21it/s, avg_epoch_loss=2.69]
INFO:root:Epoch[146] Elapsed time 9.609 seconds
INFO:root:Epoch[146] Evaluation metric 'epoch_loss'=2.687036
INFO:root:Epoch[147] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.49it/s, avg_epoch_loss=2.72]
INFO:root:Epoch[147] Elapsed time 11.151 seconds
INFO:root:Epoch[147] Evaluation metric 'epoch_loss'=2.723706
INFO:root:Epoch[148] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.65it/s, avg_epoch_loss=2.71]
INFO:root:Epoch[148] Elapsed time 10.768 seconds
INFO:root:Epoch[148] Evaluation metric 'epoch_loss'=2.713427
INFO:root:Epoch[149] Learning rate is 5e-05
100%|██████████| 50/50 [00:10<00:00,  4.72it/s, avg_epoch_loss=2.36]
INFO:root:Epoch[149] Elapsed time 10.605 seconds
INFO:root:Epoch[149] Evaluation metric 'epoch_loss'=2.358524
INFO:root:Epoch[150] Learning rate is 5e-05
100%|██████████| 50/50 [00:08<00:00,  5.60it/s, avg_epoch_loss=2.

...

100%|██████████| 50/50 [00:11<00:00,  4.43it/s, avg_epoch_loss=2.51]
INFO:root:Epoch[366] Elapsed time 11.299 seconds
INFO:root:Epoch[366] Evaluation metric 'epoch_loss'=2.510281
INFO:root:Epoch[367] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.42it/s, avg_epoch_loss=2.44]
INFO:root:Epoch[367] Elapsed time 11.313 seconds
INFO:root:Epoch[367] Evaluation metric 'epoch_loss'=2.441930
INFO:root:Epoch[368] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.51it/s, avg_epoch_loss=2.6] 
INFO:root:Epoch[368] Elapsed time 11.104 seconds
INFO:root:Epoch[368] Evaluation metric 'epoch_loss'=2.595896
INFO:root:Epoch[369] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.45it/s, avg_epoch_loss=2.73]
INFO:root:Epoch[369] Elapsed time 11.239 seconds
INFO:root:Epoch[369] Evaluation metric 'epoch_loss'=2.728650
INFO:root:Epoch[370] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.44it/s, avg_epoch_loss=2.55]
INFO:root:Epoch[370] Elapsed time 11.27

INFO:root:Epoch[402] Evaluation metric 'epoch_loss'=2.453721
INFO:root:Epoch[403] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.29it/s, avg_epoch_loss=2.9] 
INFO:root:Epoch[403] Elapsed time 11.669 seconds
INFO:root:Epoch[403] Evaluation metric 'epoch_loss'=2.899156
INFO:root:Epoch[404] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.26it/s, avg_epoch_loss=2.7] 
INFO:root:Epoch[404] Elapsed time 11.747 seconds
INFO:root:Epoch[404] Evaluation metric 'epoch_loss'=2.697285
INFO:root:Epoch[405] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.40it/s, avg_epoch_loss=2.28]
INFO:root:Epoch[405] Elapsed time 11.378 seconds
INFO:root:Epoch[405] Evaluation metric 'epoch_loss'=2.275856
INFO:root:Epoch[406] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.31it/s, avg_epoch_loss=2.35]
INFO:root:Epoch[406] Elapsed time 11.631 seconds
INFO:root:Epoch[406] Evaluation metric 'epoch_loss'=2.350699
INFO:root:Epoch[407] Learning rate is 5e-05
100

100%|██████████| 50/50 [00:11<00:00,  4.27it/s, avg_epoch_loss=2.26]
INFO:root:Epoch[439] Elapsed time 11.713 seconds
INFO:root:Epoch[439] Evaluation metric 'epoch_loss'=2.261709
INFO:root:Epoch[440] Learning rate is 5e-05
100%|██████████| 50/50 [00:12<00:00,  4.02it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[440] Elapsed time 12.455 seconds
INFO:root:Epoch[440] Evaluation metric 'epoch_loss'=2.452758
INFO:root:Epoch[441] Learning rate is 5e-05
100%|██████████| 50/50 [00:12<00:00,  4.15it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[441] Elapsed time 12.067 seconds
INFO:root:Epoch[441] Evaluation metric 'epoch_loss'=2.683148
INFO:root:Epoch[442] Learning rate is 5e-05
100%|██████████| 50/50 [00:13<00:00,  3.82it/s, avg_epoch_loss=2.6] 
INFO:root:Epoch[442] Elapsed time 13.092 seconds
INFO:root:Epoch[442] Evaluation metric 'epoch_loss'=2.604675
INFO:root:Epoch[443] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.40it/s, avg_epoch_loss=2.74]
INFO:root:Epoch[443] Elapsed time 11.35

INFO:root:Epoch[475] Evaluation metric 'epoch_loss'=2.540306
INFO:root:Epoch[476] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.45it/s, avg_epoch_loss=2.52]
INFO:root:Epoch[476] Elapsed time 11.247 seconds
INFO:root:Epoch[476] Evaluation metric 'epoch_loss'=2.522525
INFO:root:Epoch[477] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.39it/s, avg_epoch_loss=2.51]
INFO:root:Epoch[477] Elapsed time 11.397 seconds
INFO:root:Epoch[477] Evaluation metric 'epoch_loss'=2.511638
INFO:root:Epoch[478] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.48it/s, avg_epoch_loss=2.73]
INFO:root:Epoch[478] Elapsed time 11.171 seconds
INFO:root:Epoch[478] Evaluation metric 'epoch_loss'=2.734443
INFO:root:Epoch[479] Learning rate is 5e-05
100%|██████████| 50/50 [00:11<00:00,  4.43it/s, avg_epoch_loss=2.53]
INFO:root:Epoch[479] Elapsed time 11.299 seconds
INFO:root:Epoch[479] Evaluation metric 'epoch_loss'=2.534662
INFO:root:Epoch[480] Learning rate is 5e-05
100

{'Coverage[0.1]': 0.1468900966183576,
 'Coverage[0.2]': 0.21573067632850249,
 'Coverage[0.3]': 0.2784319645732688,
 'Coverage[0.4]': 0.34007648953301095,
 'Coverage[0.5]': 0.39950684380032203,
 'Coverage[0.6]': 0.452546296296296,
 'Coverage[0.7]': 0.5236010466988724,
 'Coverage[0.8]': 0.5968699677938812,
 'Coverage[0.9]': 0.6807568438003218,
 'MAE_Coverage': 0.11009236893898743,
 'MASE': 1.339614743214345,
 'MSE': 4853773.694980887,
 'MSIS': 26.626775763191397,
 'ND': 0.04708590360538351,
 'NRMSE': 0.3007756569272551,
 'QuantileLoss[0.1]': 3607258.5732551576,
 'QuantileLoss[0.2]': 4979623.119755937,
 'QuantileLoss[0.3]': 5889460.928833199,
 'QuantileLoss[0.4]': 6493021.369277573,
 'QuantileLoss[0.5]': 6853770.667743206,
 'QuantileLoss[0.6]': 6976260.657382584,
 'QuantileLoss[0.7]': 6863401.126920222,
 'QuantileLoss[0.8]': 6437612.252812195,
 'QuantileLoss[0.9]': 5554245.177227593,
 'RMSE': 2203.12816126999,
 'abs_error': 6853770.620360374,
 'abs_target_mean': 7324.822041043147,
 'abs_t

In [4]:
# seeds
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=500,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.43it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 11.292 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.72it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 10.597 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.45it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 9.172 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.98it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 10.051 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=3.771905
INFO:root:Epoch[4] Learning rate is 0.0

KeyboardInterrupt: 

In [ ]:
# seeds
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=500,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True
        
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)